In [13]:
import os
from subprocess import Popen, PIPE

PRAAT= '/usr/bin/praat'

def run_praat(script_file):
    args= [PRAAT, '--run', script_file]
    Popen(args, stdout=PIPE)

Lista de archivos de audio - concatenar audio y escribir a archivo

In [14]:
def output_audio(filenames, output_name):
    script = ""
    for f in filenames:
        script += "Read from file: \"" + f +"\" \n"

    script += "selectObject: \"Sound " + filenames[0][-6:-4] +"\" \n"
    for f in filenames[1:]:
        script += "plusObject: \"Sound " + f[-6:-4] +"\" \n"
    
    script += "Concatenate recoverably\n"
    script += "selectObject: \"Sound chain\"\n"
    script += "Write to WAV file: \"" + output_name + "\"\n"
    script += "Save as text file: \"" + output_name + ".TextGrid\"\n"
    
    
    with open("concatenar.praat", "w") as cf:
        cf.write(script)

    run_praat("concatenar.praat")

print(output_audio(["/a/la.wav", "/ja.wav", "//////////ASDadhasf/////AE.WAV"], "/homa/a/TP1/file.wav"))

FileNotFoundError: [Errno 2] No such file or directory: '/usr/bin/praat': '/usr/bin/praat'

In [1]:
import sys

diphones = ["-e", "-E", "-f", "-k", "-m", "-s", "-t", "e-", "E-",
           "s-", "ee", "eE", "Ee", "EE", "fe", "fE", "ke", "kE", 
           "me", "mE", "se", "sE", "te", "tE", "re", "rE", "ef",
           "Ef", "ek", "Ek", "em", "Em", "es", "Es", "et", "Et",
           "er", "Er", "fr", "kr", "tr", "sf", "sk", "st", "ss",
           "sm"]
prefixname = "/home/user/ph/"
wavsufix = ".wav"

def diphone_list( word ):
    diphone_list = []
    word  = "-" + word + "-"
    for i, _ in enumerate(word[:-1]):
        diphone = word[i:i+2]
        if not diphone in diphones:
            print("ERROR: invalid word. Try again.")
            sys.exit(1)
        diphone_list.append(diphone)

    return list(map(lambda s: prefixname + s + wavsufix, diphone_list))

diphone_list("exre")    

ERROR: invalid word. Try again.


SystemExit: 1

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
